In [1]:
import cv2 as cv
import threading
import time
import ipywidgets as widgets
from IPython.display import display
from gesture_grasp import Gesture_Grasp
from dofbot_utils.fps import FPS
from dofbot_utils.robot_controller import Robot_Controller

/usr/local/lib/python3.10/dist-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
robot = Robot_Controller()
robot.move_init_pose()
fps = FPS()

gesture = Gesture_Grasp()
model = 'General'

In [3]:
button_layout      = widgets.Layout(width='320px', height='60px', align_self='auto')
output = widgets.Output()

# 退出 exit
exit_button = widgets.Button(description='Exit', button_style='danger', layout=button_layout)

# 图像控件 Image widget
imgbox = widgets.Image(format='jpg', height=480, width=640)
# 垂直布局 Vertical layout
display_box = widgets.VBox([imgbox, exit_button])


Error in cpuinfo: prctl(PR_SVE_GET_VL) failed
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:
def exit_button_Callback(value):
    global model
    model = 'Exit'
    with output:
        print(model)

exit_button.on_click(exit_button_Callback)

In [5]:
def camera():
    global model, gesture
    # 打开摄像头 Open camera
    capture = cv.VideoCapture(0, cv.CAP_V4L2)
    capture.set(3, 640)
    capture.set(4, 480)
    # Be executed in loop when the camera is opened normally 
    # 当摄像头正常打开的情况下循环执行
    while capture.isOpened():
        try:
            _, img = capture.read()
            fps.update_fps()
            gesture.process(img)
            if model == 'Exit':
                capture.release()
                del gesture
                break
            fps.show_fps(img)
            imgbox.value = cv.imencode('.jpg', img)[1].tobytes()
            time.sleep(0.002)
        except Exception as e:
            print("program end")
            print(e)
            capture.release()

W0000 00:00:1744969541.074798  196591 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744969541.092912  196594 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [6]:
display(display_box,output)
threading.Thread(target=camera, ).start()

Output()

W0000 00:00:1744969542.761072  196591 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
